# Sudoku STC Solver

In [1]:
import sys
from pathlib import Path

cwd = Path.cwd()
project_root = cwd if (cwd / "stc_sudoku").is_dir() else cwd.parent
sys.path.insert(0, str(project_root))

In [2]:
import numpy as np
from stc_sudoku.funcs import run_stc_solver

In [3]:
from sudoku import Sudoku

In [4]:
def display_board(board_array, size):
    box_size = int(np.sqrt(size))
    board_list = board_array.tolist()
    puzzle = Sudoku(box_size, box_size, board=board_list)
    puzzle.show()

## 4x4 Board

In [5]:
puzzle_4x4 = np.array([
    [0, 3, 4, 0],
    [4, 0, 0, 2],
    [1, 0, 0, 3],
    [0, 2, 1, 0]
])

print("Initial 4x4 Puzzle:")
display_board(puzzle_4x4, size=4)

Initial 4x4 Puzzle:
Puzzle has exactly one solution
+-----+-----+
|   3 | 4   |
| 4   |   2 |
+-----+-----+
| 1   |   3 |
|   2 | 1   |
+-----+-----+



In [6]:
solution_4x4 = run_stc_solver(size=4, given_puzzle=puzzle_4x4, iterations=100, batch_size=500)

print("\nFinal 4x4 Output:")
display_board(solution_4x4, size=4)


Starting 4x4 Dynamic GLS STC Solver...
Step   0 | Actual Collisions:   0 | Weighted E:   0.0 | Patience: 0
>>> Perfect Solution Found! <<<

Final 4x4 Output:
Puzzle has exactly one solution
+-----+-----+
| 2 3 | 4 1 |
| 4 1 | 3 2 |
+-----+-----+
| 1 4 | 2 3 |
| 3 2 | 1 4 |
+-----+-----+



## 9x9 Board

In [7]:
# Classic 9x9 puzzle (unique solution)
puzzle_9x9 = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
], dtype=int)

print("Initial 9x9 Puzzle:")
display_board(puzzle_9x9, size=9)

Initial 9x9 Puzzle:
Puzzle has exactly one solution
+-------+-------+-------+
| 5 3   |   7   |       |
| 6     | 1 9 5 |       |
|   9 8 |       |   6   |
+-------+-------+-------+
| 8     |   6   |     3 |
| 4     | 8   3 |     1 |
| 7     |   2   |     6 |
+-------+-------+-------+
|   6   |       | 2 8   |
|       | 4 1 9 |     5 |
|       |   8   |   7 9 |
+-------+-------+-------+



In [8]:
solution_9x9 = run_stc_solver(size=9, given_puzzle=puzzle_9x9, iterations=2000, batch_size=4000)

print("\nFinal 9x9 Output:")
display_board(solution_9x9, size=9)


Starting 9x9 Dynamic GLS STC Solver...
Step   0 | Actual Collisions:  32 | Weighted E:  32.0 | Patience: 0
Step   8 | Actual Collisions:   0 | Weighted E:   0.0 | Patience: 0
>>> Perfect Solution Found! <<<

Final 9x9 Output:
Puzzle has exactly one solution
+-------+-------+-------+
| 5 3 4 | 6 7 8 | 9 1 2 |
| 6 7 2 | 1 9 5 | 3 4 8 |
| 1 9 8 | 3 4 2 | 5 6 7 |
+-------+-------+-------+
| 8 5 9 | 7 6 1 | 4 2 3 |
| 4 2 6 | 8 5 3 | 7 9 1 |
| 7 1 3 | 9 2 4 | 8 5 6 |
+-------+-------+-------+
| 9 6 1 | 5 3 7 | 2 8 4 |
| 2 8 7 | 4 1 9 | 6 3 5 |
| 3 4 5 | 2 8 6 | 1 7 9 |
+-------+-------+-------+

